In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

ANIO_INICIO = 2017
ANIO_TERMINO = 2035

COLUMNAS_POBLACION_INE = [f"{i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]

In [3]:
# Lee los casos de area de influencia y deja los diagnosticos en formato codigo
casos_area_de_influencia = pd.read_excel(
    "../data/interim/casos_teoricos_diagnosticos.xlsx", sheet_name="casos_area_de_influencia_INT"
)
casos_area_de_influencia["Diagnostico"] = (
    casos_area_de_influencia["Diagnostico"].str.split(" - ").str[0]
)
casos_area_de_influencia = casos_area_de_influencia.set_index("Diagnostico")

# Lee los porcentajes de quirurgicos y deja los diagnosticos en formato codigo
porcentajes_de_quirurgicos = pd.read_excel(
    "../data/external/incidencias_y_prevalencias_INT.xlsx", sheet_name="porcentajes_trazadoras"
)
porcentajes_de_quirurgicos["Diagnostico"] = (
    porcentajes_de_quirurgicos["Diagnostico"].str.split(" - ").str[0]
)
porcentajes_de_quirurgicos = porcentajes_de_quirurgicos.set_index("Diagnostico")
porcentajes_de_quirurgicos = porcentajes_de_quirurgicos["Porcentaje Quirúrgico"]
porcentajes_de_quirurgicos = porcentajes_de_quirurgicos.dropna()

En este caso, se multiplicaran los casos del area de influencia por el porcentaje de intervenciones quirúrgicas.

In [4]:
casos_quirurgicos = casos_area_de_influencia[COLUMNAS_POBLACION_INE].mul(
    porcentajes_de_quirurgicos, axis=0
)

In [42]:
duraciones_int_q = pd.read_excel(
    "../data/raw/5_duracion_int_q/duraciones_de_int_q_por_diagnostico.xlsx"
)

duraciones_int_q["diag_01_principal_(cod)"] = (
    duraciones_int_q["diag_01_principal_(cod)"].str.replace(".", "", regex=False).str.ljust(4, "X")
)

duraciones_int_q["diag_01_principal_(cod)"] = duraciones_int_q[
    "diag_01_principal_(cod)"
].str.replace("C341", "C34N")

duraciones_int_q = duraciones_int_q.set_index("diag_01_principal_(cod)")

# Convierte las duraciones a tiempo
COLUMNAS_DURACION = ["mean", "std", "min", "25%", "50%", "75%", "max"]
duraciones_int_q[COLUMNAS_DURACION] = duraciones_int_q[COLUMNAS_DURACION].apply(
    lambda x: pd.to_timedelta(x, unit="D")
)

duraciones_int_q = duraciones_int_q.query("ano_de_egreso == 2019")["mean"]

In [45]:
tiempo_utilizado_pabellon = casos_quirurgicos.mul(duraciones_int_q, axis=0)